#0. 프로젝트 준비

In [ ]:
# konlyp, mecab , kss install
# 설치 후 런타임 재시작
!pip install konlpy
!curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x
!pip install kss

In [112]:
# 사용할 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import math
import time
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 데이터 불러오기



In [17]:
## csv파일 불러오기
base_path = '/content/drive/MyDrive/nlp미니프로젝트/'
data = pd.read_csv(base_path + 'm0803.csv', delimiter=',', header=0)
# 불필요한 컬럼 제거
data = data.drop(['Unnamed: 0'],axis=1)
data.tail(3)

,가사
26742,현아 날 버리지 말아요 현아 날 떠나지 말아요 현아 당신 떠나시면 이 슬픔 정말 어...
26743,그때는 믿지 않았지 웃어버렸지 사랑하면서도 헤어져야 한다는 말에 아직도 믿진 않지만...
26744,단 한번 단 한번밖에 못해도 그래도 널 사랑할 수 있을까 내 전불 다 걸고 내 앞에...


In [18]:
# train, test 분리
train = data[:round(0.8*len(data))]
test = data[round(0.8*len(data)):]

print('train:',len(train)
      ,'\ntest:',len(test)
      ,'\n-----------------'
      ,'\ntrain + test:',len(train + test)
      ,'\ndata:',len(data))

train: 21396 
test: 5349 
----------------- 
train + test: 26745 
data: 26745


In [19]:
## 예제 문장 하나 불러오기
example = train.iloc[16, -1]
example

'바람에 나 얼어붙어요 빨리 나와줘 기다리게 하지 마요 차가워진 내 손을 잡아줘요 괜히 자꾸 너를 귀찮게 하는 것 같지만 Everyday 나에게 이 겨울은 말이죠 너에게 조금은 더 안길 수 있는 계절 I love you when you hold me I don t feel the cold 니가 걷는 길에 늘 서 있을게 I love you when you hold me I don t feel the cold 늘 곁에 있어줘 또 봄이 올 때까지 Woo Woo Woo I don t feel the cold Woo Woo Woo I 내 옆에 더 바짝 붙어요 눈이 내려요 감기 걸린 것 같아요 차가워진 나를 좀 안아줘요 괜히 자꾸 너를 귀찮게 하는 것 같지만 Everyday 나에게 이 겨울은 말이죠 내가 그댈 많이 안아 줄 수 있는 계절 I love you when you hold me I don t feel the cold 니가 걷는 길에 늘 서 있을게 I love you when you hold me I don t feel the cold 늘 곁에 있어줘 또 봄이 올 때까지 얼어붙은 모든 게 다 녹아 내리죠 니가 웃을 때 나를 안아줄 때 니가 걷는 길에 늘 서 있을게 I love you when you hold me I don t feel the cold 늘 곁에 있어줘 또 봄이 올 때까지 Woo Woo Woo I don t feel the cold Woo Woo Woo I Woo Woo Woo I don t feel the cold Woo Woo Woo I'

#2. 데이터 전처리
 - 1) 토큰화
 - 2) POS 태깅
 - 3) 불용어 처리



In [20]:
# csv파일 병합 파일에서 1차적으로 특수문자 제거 완료

# 정규 표현식으로 데이터 정제
# 한글과 공백을 제외하고 모두 제거
train['가사'] = train['가사'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,가사
0,그리운 만큼만 기억해 다시 널 만날 수 없게 미안해 붙잡지 않을게 비라도 내리면 좋...
1,바쁘게 지냈었어 밀린 잠도 몰아서 잤어 티빌 켜고 못 봤던 것도 다 보고 약속도 잡...
2,낯선 너를 보면서 지는 해가 서러워 너무 다른 너와 나 여기까지도 힘들었구나 행복이...
3,길을 잃은 작은 아이 시간이 멈춘 그 곳에서 맘을 잃은 작은 아이 너와 걷던 그 거...
4,잘 지내고 있는 거겠죠 좀처럼 연락이 없네요 요즘 많이 바쁜 가 봐요 이미 다른 사...
...,...
21391,노란 불빛들이 나를 따라오는 밤 딱히 원했던 건 아냐 당연하지만 내가 선택하지 않은...
21392,쉽게 잠이 들지 못한 밤이 늘었어 평소보다 조금 일찍 나와서 별다를거 없는 하룰 시...
21393,아무도 너를 묻지 않아 누구도 위로하지 않아 시간이 지나서 괜찮아진 나에게 어둠이 ...
21394,삶의 무게에 지쳐있는 너 잘못된 선택에 아파하고 있을 때 그저 옆에 있는 것 말곤 ...


In [22]:
train.iloc[16, -1]

'바람에 나 얼어붙어요 빨리 나와줘 기다리게 하지 마요 차가워진 내 손을 잡아줘요 괜히 자꾸 너를 귀찮게 하는 것 같지만  나에게 이 겨울은 말이죠 너에게 조금은 더 안길 수 있는 계절              니가 걷는 길에 늘 서 있을게              늘 곁에 있어줘 또 봄이 올 때까지              내 옆에 더 바짝 붙어요 눈이 내려요 감기 걸린 것 같아요 차가워진 나를 좀 안아줘요 괜히 자꾸 너를 귀찮게 하는 것 같지만  나에게 이 겨울은 말이죠 내가 그댈 많이 안아 줄 수 있는 계절              니가 걷는 길에 늘 서 있을게              늘 곁에 있어줘 또 봄이 올 때까지 얼어붙은 모든 게 다 녹아 내리죠 니가 웃을 때 나를 안아줄 때 니가 걷는 길에 늘 서 있을게              늘 곁에 있어줘 또 봄이 올 때까지                          '

## 1) 토큰화

In [7]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator # 토큰의 목록 또는 반복자를 받는 내장 팩토리 함수

#한글 문장 토큰화 하기 위해 필요 (korean sentence splitter)
import kss

# konlpy에서 라이브러리 불러오기
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Komoran
from konlpy.tag import Okt
from konlpy.tag import Mecab

In [8]:
# 형태소 분석을 위한 객체 생성
# 꼬꼬마
#kkma = Kkma()

# 코모란
#komoran = Komoran()

# # 한나눔
#hannanum = Hannanum()   # 너무 느려서 사용 안함

# Okt
#okt = Okt()

# Mecab                    
mecab = Mecab()

In [ ]:
# 토큰화 함수 : 토큰 타입을 변경하고 싶으면 return 부분 수정  
# 품사 태깅 퀄리티: 꼬꼬마 > 코모란 > Okt            # .morphs는 형태소만 추출   .pos는 형태소 + 품사 모두 추출   .nouns는 명사 추출
mecab.morphs(train['가사'][4])

In [ ]:
# 토큰화된 가사 저장
token_1 = []

# 불용어가 추가된 형태소 분석기
for sentence in train['가사']:
  try:
    # 토큰화
    temp = mecab.morphs(sentence)
    token_1.append(temp)
  except:
    token_1.append('x')
token_1

## 2) POS 태깅

In [ ]:
temp = mecab.pos(sentence)
temp

In [ ]:
# 토큰화된 가사 저장
token = []

# 불용어가 추가된 형태소 분석기
for sentence in train['가사']:
  # 토큰화
  temp = mecab.pos(sentence)
  token.append(temp)

token

## 3) 불용어 처리

In [29]:
stopword = []

for i in range(len(token)):
  for j in range(len(token[i])):
    word, tag = token[i][j]
    # Mecab 품사 태그 기준
    # SF, SP, SS, SE, SO, SW: 각종 부호들, SL: 외국어, SH: 한자, SN: 숫자
    if tag == 'SF' or tag == 'SP' or tag == 'SS' or tag == 'SE' or tag == 'SO' or tag == 'SW' or tag == 'SL' or tag == 'SH' or tag == 'SN':
      stopword.append(word)

display(stopword)

# 애초에 잘 정제 된 노래 가사이다 보니 특수 기호, 부호들이 없었다.
# 그나마 영어 가사 마저도 데이터 전처리 단계에서 날려버렸..

# 추후에 회의를 통해 추가할 품사가 있는지 검토

[]

In [30]:
## 불용어 처리 된 테깅
# 토큰화된 가사 저장
tagging_with_sw = []

# 불용어가 추가된 형태소 분석기
for sentence in train['가사']:
  # 토큰화
  temp = mecab.morphs(sentence)
  # 불용어 제거
  temp = [word for word in temp if word not in stopword] 
  tagging_with_sw.append(temp)


train['불용어처리 된 가사 토큰'] = tagging_with_sw
train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,가사,불용어처리 된 가사 토큰
0,그리운 만큼만 기억해 다시 널 만날 수 없게 미안해 붙잡지 않을게 비라도 내리면 좋...,"[그리운, 만큼, 만, 기억, 해, 다시, 널, 만날, 수, 없, 게, 미안, 해,..."
1,바쁘게 지냈었어 밀린 잠도 몰아서 잤어 티빌 켜고 못 봤던 것도 다 보고 약속도 잡...,"[바쁘, 게, 지냈었, 어, 밀린, 잠, 도, 몰, 아서, 잤, 어, 티, 빌, 켜..."
2,낯선 너를 보면서 지는 해가 서러워 너무 다른 너와 나 여기까지도 힘들었구나 행복이...,"[낯선, 너, 를, 보, 면서, 지, 는, 해, 가, 서러워, 너무, 다른, 너, ..."
3,길을 잃은 작은 아이 시간이 멈춘 그 곳에서 맘을 잃은 작은 아이 너와 걷던 그 거...,"[길, 을, 잃, 은, 작, 은, 아이, 시간, 이, 멈춘, 그, 곳, 에서, 맘,..."
4,잘 지내고 있는 거겠죠 좀처럼 연락이 없네요 요즘 많이 바쁜 가 봐요 이미 다른 사...,"[잘, 지내, 고, 있, 는, 거, 겠, 죠, 좀처럼, 연락, 이, 없, 네요, 요..."
...,...,...
21391,노란 불빛들이 나를 따라오는 밤 딱히 원했던 건 아냐 당연하지만 내가 선택하지 않은...,"[노란, 불빛, 들, 이, 나, 를, 따라오, 는, 밤, 딱히, 원했, 던, 건, ..."
21392,쉽게 잠이 들지 못한 밤이 늘었어 평소보다 조금 일찍 나와서 별다를거 없는 하룰 시...,"[쉽, 게, 잠, 이, 들, 지, 못한, 밤, 이, 늘, 었, 어, 평소, 보다, ..."
21393,아무도 너를 묻지 않아 누구도 위로하지 않아 시간이 지나서 괜찮아진 나에게 어둠이 ...,"[아무, 도, 너, 를, 묻, 지, 않, 아, 누구, 도, 위로, 하, 지, 않, ..."
21394,삶의 무게에 지쳐있는 너 잘못된 선택에 아파하고 있을 때 그저 옆에 있는 것 말곤 ...,"[삶, 의, 무게, 에, 지쳐, 있, 는, 너, 잘못, 된, 선택, 에, 아파하, ..."


# 3. 감정 사전 구축
 - Word2Vec을 사용한 단어 학습 (CBOW 사용)



In [116]:
from gensim.models.word2vec import Word2Vec as wv


model_mecab = wv(tagging_with_sw,    # 테깅 데이터를 담은 변수
                 min_count = 4,      # 데이터에 등장하는 단어의 최소 빈도 수 제한
                 size = 128,         # 벡터 스페이스(차원 크기) 설정
                 iter = 20,          # 학습 반복 횟수 (= 딥러닝의 epoch)
                 sg = 0,             # 0이면 CBOW, 1이면 skip-gram 사용
                 batch_words = 1024, # 사전을 구축할 때 한번에 읽을 단어 수
                 window = 5,         # 모델 학습 시 앞뒤로 읽을 단어 수
                 alpha = 0.025,      # learning rate  *디폴트 값 추천
                 negative = 5        # negative samples의 개수 *디폴트 값 추천(클수록 학습속도 느려짐, 너무 작으면 학습 X)
                 )

In [136]:
# 입력된 단어와 가장 유사한 단어 추출
def similarity(x,y):
  #유사 단어 검색 및 추가
  vocab = model_mecab.most_similar(x, topn = y) # 추출할 유사 단어 개수

  similarity_word = pd.DataFrame(vocab)
  similarity_word.columns = ['유사 단어','유사도']

  display(similarity_word)
  print(f'ㄴ "{x}"와(과) 유사한 단어')

In [137]:
similarity('우주',20) # '알고 싶은 단어', 알고 싶은 단어 개수 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


,유사 단어,유사도
0,지구,0.585647
1,섬,0.585530
2,은하수,0.571213
3,숲,0.550770
4,성과,0.528016
5,도시,0.526761
6,바다,0.524379
7,빌딩,0.521112
8,밤하늘,0.516110
9,별,0.498756


ㄴ "우주"와(과) 유사한 단어


< 유사단어 정리 표 >

||사랑|이별|우정|희망|설렘|
|--|--|--|--|--|--|
|1|행복	0.528893|헤어짐	0.570216|제약	0.586112|소망	0.596384|설레임	0.723407
|2|원망	0.518756|만남	0.490854|계피	0.554139|믿음	0.577792|떨림	0.657539
|3|미안	0.494765|인연	0.475418|을려나	0.540440|바램	0.556808|느낌	0.564502
|4|용서	0.491051|사랑	0.464315|우직	0.538465|절망	0.550740|체온	0.523427
|5|이해	0.471536|헤어져야	0.444108|깔쌈	0.522884|시련	0.544343|감정	0.516987
|6|이별	0.464315|현실	0.415012|꽃줄기	0.513335|두려움	0.506998|속삭임	0.513355
|7|말	  0.463434|고통	0.414333|빚어진	0.495641|움직임	0.493009|끌림	0.495517
|8|부족	0.463403|착각	0.394413|죽음	0.488032|어려움	0.473137|통증	0.490577
|9|결혼	0.459441| 아픔	0.384028|졸업앨범	0.483467|화분	0.462543|두근거림	0.484812
|10|약속 0.458218|헤어져	0.376740|바달	0.482132|고난	0.460396|감동	0.467665

> 입력한 파라미터
 - CBOW (sg = 0)
 - min_count = 4    
 - size = 128      
 - iter = 20
 - batch_words = 1024
 - window = 5
 - alpha = 0.025
 - negative = 5


In [141]:
# 사전 구축 알고리즘
lyric_emotion = ['행복','사랑','슬픔','이별','설렘','희망']
lyric_emotions = []

for i in range(len(lyric_emotion)):
  selected_emotion = []
  for word in model_mecab.most_similar(lyric_emotion[i],topn = 50): # 필요에 따라 topn 설정하기!
    if(word[1] >= 0.5): # word[1]은 유사도 부분
      selected_emotion.append(word)
  lyric_emotions.append(selected_emotion)
  
lyric_emotions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


[[('불행', 0.7112571001052856),
  ('소중', 0.6398675441741943),
  ('충분', 0.5880572199821472),
  ('사랑', 0.5829553604125977),
  ('감사', 0.5663818717002869),
  ('함께', 0.5605239868164062),
  ('불쌍', 0.5441039204597473),
  ('완벽', 0.5315811634063721),
  ('편안', 0.5235249996185303),
  ('부족', 0.5229876637458801),
  ('특별', 0.5129358768463135)],
 [('행복', 0.5829553604125977),
  ('미안', 0.5551435947418213),
  ('용서', 0.5459126234054565),
  ('약속', 0.5028866529464722)],
 [('아픔', 0.7432297468185425),
  ('외로움', 0.6080388426780701),
  ('그리움', 0.5690194964408875),
  ('미움', 0.5656317472457886),
  ('고통', 0.5154553651809692),
  ('절망', 0.502477765083313)],
 [('헤어짐', 0.5901002287864685)],
 [('설레임', 0.6741498112678528),
  ('떨림', 0.6056473255157471),
  ('감정', 0.512484073638916),
  ('느낌', 0.5013058185577393)],
 [('소망', 0.6277005672454834), ('믿음', 0.5503544807434082)]]

#4. 감정 분류
 - 1) 단어 임베딩
 - 2) LSTM을 활용한 감정 분류 모델 구축

 - Word2Vec(skip-gram or **LSTM**)을 사용 



# 5. 감정 유사도 계산
 - 단어 간 거리 유사도 판별 (Word2Vec 임베딩 모델 사용)



6. 실험 평가
 - 1) Word2Vec 모델 바교
 - 2) 문맥 의미 파악
 - 3) 모델 비교 실험
    - SVM
    - Logistic Regression
    - LSTM
 - 4) 신규 가사 감정 테스트
 - 5) 기존 연구와 비교분